In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import shutil, os, random, gc, time, traceback
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras import backend as K

from keras.applications import InceptionV3, VGG16, ResNet50, InceptionResNetV2, DenseNet201

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.losses import binary_crossentropy, categorical_crossentropy

import cache_magic
%matplotlib inline

Using TensorFlow backend.


In [2]:
pd.options.display.float_format = "{:,.2f}".format

DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data')).replace('\\', '/') + '/'
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'model_coarse')).replace('\\', '/') + '/'
BEST_MODEL = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'model','VGG16_data_imbalance.h5')).replace('\\', '/') 

NUM_CLASSES = 1000
ATTR_TYPES = {'1':"Estampado",'2':"Tejido",'3':"Forma",'4':"Partes",'5':"Estilo"}

In [3]:
# Rutas a directorios de datos
ANNO_COARSE_PATH = DATA_PATH + 'anno_coarse'
TRAIN_PATH = ANNO_COARSE_PATH + '/train'
VALIDATION_PATH = ANNO_COARSE_PATH + '/val'
TEST_PATH = ANNO_COARSE_PATH + '/test'
AUGMENTED_DATA_PATH = ANNO_COARSE_PATH + '/augmented'
IMG = DATA_PATH + 'img'

# Rutas a ficheros de datos
ATTR_CLOTH_LIST_FILE = ANNO_COARSE_PATH + '/list_attr_cloth.txt'
ATTR_IMG_LIST_FILE = ANNO_COARSE_PATH + '/list_attr_img.txt'
PARTITION_FILE = ANNO_COARSE_PATH + '/list_eval_partition.txt'

TRAIN_FILE_PATH = ANNO_COARSE_PATH + "/train_attr.txt"

# 1. Código de pre-procesamiento y entrenamiento

A continuación se define el código necesario para el procesamiento de los ficheros de datos, así como para el entrenamiento de los modelos. El código, salvo pequeños cambios por la estructura de datos, es idéntico al empleado para la versión reducida del conjunto de datos por lo que se omite su descripción detallada. 

## 1.1 Pre-procesamiento de datos

In [4]:
def read_partition_files():
    data = {}
    data['train'] = []
    data['test'] = []
    data['val'] = []
    with open(PARTITION_FILE) as fp:
        fp.readline() # Ingorar numero de etiquetas
        fp.readline() # Ingorar numero de etiquetas
        for line in fp:
            fields = line.split()
            
            img_path = fields[0]
            img_path = DATA_PATH + img_path
            
            partition = fields[1]
            data[partition].append(img_path)
    return data

In [5]:
partition_images = read_partition_files()

In [6]:
# Lectura de la lista de atributos (fichero list_attr_cloth.txt)
def read_attr_cloth_list():
    column_to_attr_name = {}
    attr_type_to_columns = {}
    
    with open(ATTR_CLOTH_LIST_FILE) as fp: 
        fp.readline() # Ingorar numero de etiquetas
        fp.readline() # Ignorar cabecera
        column = 0
        for line in fp: 
            fields = line.split()
            attr = " ".join(fields[:-1])
            attr_type = fields[-1]
            column_to_attr_name[column] = attr
            if attr_type in attr_type_to_columns:
                attr_type_to_columns[attr_type].append(column)
            else:
                attr_type_to_columns[attr_type] = [column]
            column += 1
    return column_to_attr_name, attr_type_to_columns

In [7]:
# column_to_attr_name: Mapping número columna -> Nombre de atributo
# attr_type_to_columns: Mapping tipo de atributo - Lista de columnas de atributos de dicho tipo
column_to_attr_name, attr_type_to_columns = read_attr_cloth_list()           
# Listado con los nombre de los atributos
attributes = list(column_to_attr_name.values())

In [8]:
# Devuelve el listado de imágnes del fichero recibido como parámetro
def read_image_list(file_name):
    file_path = ANNO_COARSE_PATH + '/' + file_name + '.txt'
    with open(file_path) as fp:
        return [DATA_PATH + img.rstrip('\n') for img in fp]
    
# Devuelve el path de la partición a partir del nombre de la partición
def build_partition_data_path(partition):
    return ANNO_COARSE_PATH + '/' + partition

# Creación de la estructura de datos necesaria para los generadores de aumentación de datos
def create_data_structure(partition_images, force=False):
    for partition in ['test', 'train', 'val']:
        partition_data_path = build_partition_data_path(partition)
        
        # Forzar recarga
        if os.path.exists(partition_data_path) and force:
            shutil.rmtree(partition_data_path)
        
        if not os.path.exists(partition_data_path):
            os.mkdir(partition_data_path)
            # Carga del listado de imágenes
            image_list = partition_images[partition]
            # Copia
            for i, src in enumerate(image_list):
                dst_filename = src.replace('../data/img/', '')
                dst_filename = dst_filename.replace('/','')
                dst = partition_data_path + '/' + dst_filename
                shutil.copyfile(src, dst)


In [9]:
create_data_structure(partition_images, False)

## 1.2 Aumentación de datos:

In [10]:
# Construcción del dataframe requerido para usar la función flow_from_dataframe
# En el caso de no pasarle attr_type, tiene en cuenta todos los atributos (empleado en la versión con un único modelo)
# En caso contrario, filtra los atributos pertenecientes al tipo pasado por parámetro. (empleado en la versión de un modelo por
# tipo de atributo)
def build_iterator_dataframe(partition, attr_type=None):
    partition_attribute_filepath = ANNO_COARSE_PATH + "/" + partition + "_attr.txt"
    names = []
    names.append('filename')
    names.extend(attributes)
    data = pd.read_csv(partition_attribute_filepath, sep = ' ', names = names, index_col=False)
    
    # Filtrado por tipo de atributo. Empleado para los clasificadores separados por tipo de atributo. 
    if attr_type is not None:
        columns = attr_type_to_columns[str(attr_type)]
        fst, lst = columns[0], columns[-1]+1
        data = data.iloc[:, fst:lst]
    
    return data

# Devuelve los nombres de los atributos pertenecientes al tipo de atributo pasado por parámetro
def get_attributes_for_attribute_type(attr_type):
    if attr_type is None:
        return attributes
    else:
        return [column_to_attr_name[c] for c in attr_type_to_columns[str(attr_type)]]
    
SEED = 1
BATCH_SIZE = 4


# Construcción del generador de datos de entrenamiento
def build_train_datagen(shear_range=0.025, zoom_range=0.025, rotation_range=2.5, fill_mode='nearest'):
    return ImageDataGenerator(
        rescale=1/255.0,
        shear_range=shear_range,
        zoom_range=zoom_range,
        rotation_range=rotation_range,
        fill_mode = fill_mode,
        horizontal_flip=True)

# Construcción de los generadores de datos de entrenamiento y validación
def build_generators(img_size, shear_range, zoom_range, rotation_range, fill_mode, attr_type = None):
        
    # Borrar los directorios con las imagenes generadas
    if os.path.exists(AUGMENTED_DATA_PATH):
        shutil.rmtree(AUGMENTED_DATA_PATH)
    os.mkdir(AUGMENTED_DATA_PATH)
    
    # Generator de entrenamiento
    train_datagen = build_train_datagen(shear_range, zoom_range, rotation_range, fill_mode)
    train_generator = train_datagen.flow_from_dataframe(
        dataframe = build_iterator_dataframe('train', attr_type),
        x_col="filename",
        y_col=get_attributes_for_attribute_type(attr_type),
        directory=TRAIN_PATH,
        target_size=(img_size,img_size),
        batch_size=BATCH_SIZE,
        class_mode='raw',
        color_mode='rgb',
        seed=SEED,
        save_to_dir=AUGMENTED_DATA_PATH
    )
    
    print(TRAIN_PATH)

    # Generator de validación
    validation_datagen = ImageDataGenerator(
         rescale=1./255
    )

    validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=build_iterator_dataframe('val', attr_type),
        x_col="filename",
        y_col=get_attributes_for_attribute_type(attr_type),
        directory=VALIDATION_PATH,
        target_size=(img_size,img_size),
        #batch_size=BATCH_SIZE,
        class_mode='raw',
        color_mode='rgb',
        seed=SEED
    )
    
    return train_generator, validation_generator


# Construcción del generador de datos de test
def build_test_generator(img_size=224, attr_type = None):
        
    test_datagen = ImageDataGenerator(
         rescale=1./255
    )
    
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=build_iterator_dataframe('test', attr_type),
        x_col="filename",
        y_col=get_attributes_for_attribute_type(attr_type),        
        directory=TEST_PATH,
        target_size=(img_size,img_size),
       # batch_size=BATCH_SIZE,
        class_mode=None,
        color_mode='rgb',
        seed=SEED,
        shuffle=False
    )
    
    return test_generator 

## 1.3 Automatización del entrenamiento

In [11]:
EARLY_STOPPING_PATIENCE = 8
LR_REDUCTION_PATIENCE = 4

# Conversión del tamaño de la imagen al shape esperado como input del modelo
def to_input_shape(img_size):
    return (img_size,img_size,3)

# Construcción del path donde se persiste el modelo
def to_model_path(model_name, model_id, attr_type = None):
    if not os.path.exists(MODEL_PATH):
        os.mkdir(MODEL_PATH)
    if attr_type is None:
        return '{}/{}_{}.h5'.format(MODEL_PATH, model_name, model_id)
    else:
        return '{}/{}_{}_{}.h5'.format(MODEL_PATH, model_name, model_id, attr_type)

# Construcción del nombre de la función de instanciación de un modelo
def to_build_function(model_name):
    return 'build_' + model_name

# Desactiva el entrenamiento del X% de capas iniciales. 
def freeze_layers(model, ratio=0.5):
    n_layers = len(model.layers)
    last_layer_to_freeze = int(n_layers*ratio)
    
    for layer in model.layers[:last_layer_to_freeze]:
        layer.trainable = False
    for layer in model.layers[last_layer_to_freeze:]:
        layer.trainable = True
    
    return model

def freeze_all_layers(model):
    for layer in model.layers:
        layer.trainable = False

def build_callbacks(model_name, model_id, store_model, attr_type):
    callbacks = []
    # Parar la ejecución si el loss no disminuye en 3 iteraciones
    early_stopping = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, verbose=1, min_delta=1e-4)
    callbacks.append(early_stopping)
    # Reducir el learning rate si el loss no disminuye en 3 iteraciones
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_REDUCTION_PATIENCE, verbose=1, min_delta=1e-4)  
    callbacks.append(reduce_lr)
    # Guardar el modelo que más disminuye el loss en fichero
    if store_model:
        checkpoint = ModelCheckpoint(filepath = to_model_path(model_name, model_id, attr_type), 
                                     monitor='val_auc', verbose=1, save_best_only=True, mode='max')
        callbacks.append(checkpoint)
    return callbacks

# Entrenamiento, y todo el pre-procesamiento necesario. 
def train(model_name,
          model_id=None,
          max_epochs=1000, 
          img_size=224,
          shear_range=0.05, 
          zoom_range=0.150, 
          rotation_range=2.5, 
          fill_mode='constant',
          store_model=True,
          freeze_layers_ratio=None,
          loss_function=binary_crossentropy,
          attr_type=None,
          class_weight = None
         ):
    
    # Forzar limpieza y pasar el GC
    K.clear_session()
    #cuda.select_device(0)
    #cuda.close()
    gc.collect()
    
    # Instanciación del modelo. Todos los métodos siguen el patrón "build_xxx", donde xxx es esl nombre del modelo
    input_shape = to_input_shape(img_size)
    model = globals()[to_build_function(model_name)]()
    #model.summary()
    
    # COMPILE con run options para debug en caso de oom
    #run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)
    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', loss=loss_function, metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
    # Instanciación de los generators de datos
    train_generator, validation_generator = build_generators(img_size, shear_range, 
                                                             zoom_range, rotation_range, fill_mode, attr_type)
    
    
    h = model.fit_generator(train_generator, validation_data=validation_generator, 
                            epochs=max_epochs, callbacks = build_callbacks(model_name, model_id, False, attr_type),
                            steps_per_epoch=5000,
                            validation_steps=1000
                           )
    # Segunda fase de training para entrenar el resto de las capas
    model = freeze_layers(model, freeze_layers_ratio)
    model.compile(optimizer='adam', loss=loss_function, metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    h = model.fit_generator(train_generator, validation_data=validation_generator, 
                            epochs=max_epochs, callbacks = build_callbacks(model_name, model_id, store_model, attr_type),
                            steps_per_epoch=5000,
                            validation_steps=1000,
                            class_weight=class_weight
                           )

    
    return (h.history['val_loss'], h.history['val_accuracy'], h.history['val_auc'])

## 1.3 Automatización de la evaluación

In [12]:
KS =  [3,5,20]

# Calcula
## Número de veces que aparece un atributo en una predicción
## Número de veces en el que la predicción de un atributo se realiza de forma correcta
## Número de veces en el que la predicción de un atributo se realiza de forma incorrecta
def get_prediction_metrics(y_true_df, y_pred, k=6):
    
    # Inicialización de los vectores
    times_predicted = [0] * NUM_CLASSES
    ground_truth = [0] * NUM_CLASSES
    times_correctly_predicted = [0] * NUM_CLASSES
    times_incorrectly_predicted = [0] * NUM_CLASSES

    for index, row in y_true_df.iterrows():
        
        # Obtención de los verdaderos positivos
        true_possitives = np.where(row==1)
        for i in true_possitives[0]:
            ground_truth[i]+=1

        # Obtención de los k valores con mayor probabilidad
        predicted_k_values = np.argpartition(y_pred[index], -k)[-k:]
        for p in predicted_k_values:
            
            # Clasificación
            times_predicted[p]+=1
            if p in true_possitives[0]:
                times_correctly_predicted[p]+=1
            else:
                times_incorrectly_predicted[p]+=1
    return times_predicted, ground_truth, times_correctly_predicted, times_incorrectly_predicted

# Calcula recall, precision y f1 usando macro averaging
def compute_macro_average_metrics(times_predicted, ground_truth, tp, fp):
    m = {}
    recall_per_label = [a/max(1,b) for a,b in zip(tp,ground_truth)]
    precision_per_label = [a/max(1,b) for a,b in zip(tp,times_predicted)]
    m['recall'] =  sum(recall_per_label)/NUM_CLASSES
    m['precision'] = sum(precision_per_label)/NUM_CLASSES
    f1_per_label = [2 * (p * r) / max(1,(p + r)) for p,r in zip(precision_per_label, recall_per_label)]
    m['f1'] = sum(f1_per_label)/NUM_CLASSES
    return m

# Calcula recall, precision y f1 usando micro averaging
def compute_micro_average_metrics(times_predicted, ground_truth, tp, fp):
    m = {}
    m['recall'] = sum(tp)/sum(ground_truth)
    m['precision'] = sum(tp)/sum(times_predicted)
    m['f1'] = 2 * (m['precision'] * m['recall']) / max(1,(m['precision'] + m['recall']))
    return m

# Calcula recall, precision y f1 usando micro/macro averaging para las k etiquetas con mayor probabilidad
def compute_evaluation_results(y_true_df, y_pred):
    results = {}
    for k in KS:
        # Calculo métricas de clasificación
        times_predicted, ground_truth, tp, fp = get_prediction_metrics(y_true_df, y_pred, k)
        
        results_k = {}
        results[k] = results_k
        # Micro average
        results_k['micro'] = compute_micro_average_metrics(times_predicted, ground_truth, tp, fp)
        
        # Macro average
        results_k['macro'] = compute_macro_average_metrics(times_predicted, ground_truth, tp, fp)
    return results

# Evaluación de un modelo.     
def evaluate(model_name, model_id):
    # Forzar limpieza y pasar el GC
    K.clear_session()
    gc.collect()

    # Recuperar el modelo guardado
    model = load_model(to_model_path(model_name, model_id), compile = False)
    model.compile(optimizer='adam', loss=binary_crossentropy, metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
    # Generator de datos de test
    test_generator = build_test_generator()
    
    # Predicción
    y_pred = model.predict_generator(test_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1) 

    # Carga de las etiquetas del conjunto de test
    partition_attribute_filepath = ANNO_COARSE_PATH + "/test_attr.txt"
    y_true_df = pd.read_csv(partition_attribute_filepath, sep = ' ', names = [str(i) for i in range(0,NUM_CLASSES)], index_col=False)
    
    # Calcula resultados en formato r[k][average type][metric] donde
    # k - numero de etiquetas con mayor probabilidad
    # average type - micro|macro
    # metric - precision|recall|f1
    results = compute_evaluation_results(y_true_df, y_pred)
        
    return (y_pred, results)

In [13]:
COLUMNS = ['recall.micro','precision.micro','f1.micro','recall.macro','precision.macro','f1.macro']

def build_result_df(result_list, k):
    rows = []
    names = []
    for model_results, model_name in result_list:
        row = []
        rows.append(row)
        names.append(model_name)
        for avg in ['micro', 'macro']:
            for metric in ['recall', 'precision', 'f1']:
                row.append(model_results[k][avg][metric])

    return pd.DataFrame(rows, columns=COLUMNS, index=names)

In [14]:
# Construcción de la tabla de resultados a partir de la lista de listas de valores predichos por un modelo sobre la partición de test
def build_results_table(y_pred):

    # Carga de las anotaciones reales de la partición de test
    partition_attribute_filepath = ANNO_COARSE_PATH + "/test_attr.txt"
    y_true_df = pd.read_csv(partition_attribute_filepath, sep = ' ', names = [str(i) for i in range(0,NUM_CLASSES)], index_col=False)

    # Columna de tipo de atributo
    attr_types = []
    for k, v in attr_type_to_columns.items():
        attr_types.extend([ATTR_TYPES[str(k)]] * len(v))

    # Columna de frecuencia de predicción, Verdaderos positivos y Falsos positivos
    times_predicted, ground_truth, times_correctly_predicted, times_incorrectly_predicted = get_prediction_metrics(y_true_df, y_pred)

    # Columnas de sensibilidad y precisión
    recall = [a/max(1,b) for a,b in zip(times_correctly_predicted, ground_truth)]
    precision = [a/max(1,b) for a,b in zip(times_correctly_predicted, times_predicted)]
    f1 = [2 * (p * r) / max(1,(p + r)) for p,r in zip(precision, recall)]

    # Creación de dataframe
    d = {'Atributo': attributes, 'Tipo': attr_types, 'Frecuencia real' : ground_truth, 
         'Frecuencia predicción' : times_predicted, 'TP': times_correctly_predicted, 'FP' : times_incorrectly_predicted, 
         'Recall' : recall, 'Precision' : precision, 'F1' : f1}
    return pd.DataFrame(data=d)


def build_results_table_by_attr_type(results):
    df = build_results_table(results).groupby('Tipo').mean()
    return df.drop(columns=['Frecuencia real', 'Frecuencia predicción', 'TP', 'FP'])

# 2. Modelo con un único clasificador

En este apartado se va a entrenar un modelo basado en un único clasificador para la clasificación de los atributos. Se emplea como base el clasificador con el que se ha obtenido los mejores resultados con el conjunto reducido de datos para beneficiarse del *transfer learning*. Recordemos que este modelo estaba entrenado a partir de la arquitectura VGG16.

Para ello se importa el modelo y se reemplaza la capa de salida por una con 1000 neuronas. Se emplea *sigmoid* como función de activación en la capa de salida para que el clasificador sea multi-etiqueta. 

In [15]:
def build_VGG16():
    loaded_model = load_model(BEST_MODEL, compile = False)
    output = Dense(NUM_CLASSES, activation='sigmoid', name='dense_2')(loaded_model.layers[-2].output)
    model = Model(inputs=loaded_model.input, outputs=output)
    
    return model


Entrenamos el modelo siguiendo los mismos pasos que se siguieron para entrenar el modelo original:
1. Se entrena primero las capas densas añadidas al final de la red
2. Se entrena el resto de la red, manteniendo fijos los pesos del 80% de capas iniciales. Este valor es con el que se obtuvo mejores resultados con el conjunto de datos reducido. 

In [16]:
#%cache -r coarse_vgg16_res
%cache coarse_vgg16_res = train('VGG16',model_id='coarse', freeze_layers_ratio=0.80)

Loading cached value for variable 'coarse_vgg16_res'. Time since caching: 9 days, 5:29:46.530070


Y lo evaluamos con la partición de test. Para la evaluación se emplea k=5, esto es, se calcula la precisión, exhaustividad y F1 en base a los 5 atributos con mayor probabilidad en el output del modelo. 

En este caso, al no existir un número fijo de atributos asociado a una observación, la métrica más robusta es la exhaustividad:

**Precisión** = Verdaderos positivos / (Verdaderos positivos + Falsos positivos)

**Exhaustividad** = Verdaderos positivos / (Verdaderos positivos + Falsos negativos) 

En este caso la precisión (y por tanto, también el F1) no es una métrica robusta puesto que existen observaciones asociadas a un número de atributos inferior a K. En esos casos, incluso en el caso de que todos los atributos se identifiquen de forma correcta, existirá un número de falsos positivos equivalente a la diferencia entre K y el número real de atributos positivos. La exhaustividad emplea en el denominador el *ground truth* y no depende del valor de K elegido.

El valor elegido para K es 5 puesto que éste es el que se emplea en los *papers* del estado del arte para establecer una comparativa entre modelos. 

In [17]:
#%cache -r vgg16_coarse_test_res
%cache vgg16_coarse_test_res = evaluate('VGG16', model_id = 'coarse')

Loading cached value for variable 'vgg16_coarse_test_res'. Time since caching: 9 days, 5:21:21.831215


In [18]:
build_result_df([(vgg16_coarse_test_res[1], 'VGG16')], k=5)

,recall.micro,precision.micro,f1.micro,recall.macro,precision.macro,f1.macro
VGG16,0.01,0.01,0.00,0.01,0.00,0.00


In [19]:
r = build_results_table(vgg16_coarse_test_res[0])
#r.to_csv('results.csv', index=True)  

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(r)

                      Atributo       Tipo  Frecuencia real  \
0                       a-line      Forma                0   
1                     abstract      Forma              418   
2             abstract chevron      Forma              767   
3       abstract chevron print      Forma               16   
4             abstract diamond      Forma                6   
5              abstract floral      Forma                5   
6        abstract floral print      Forma               53   
7                 abstract geo      Forma               25   
8           abstract geo print      Forma               31   
9             abstract paisley      Forma               17   
10            abstract pattern      Forma               14   
11              abstract print      Forma               26   
12            abstract printed      Forma              289   
13             abstract stripe      Forma               23   
14                        acid      Forma               14   
15      

El valor de *top-5 recall* es 0.01, muy inferior al 0.54 que obtiene *FashionNet*, el modelo proporcionado junto con el dataset de *DeepFashion*.

## 2.1 Modelos alternativos

Dado el pobre rendimiento obtenido con el clasificador anterior se probará varias alternativas para probar si con una simple red neuronal convolucional es posible obtener un rendimiento superior. Definimos el código necesario para estas versiones alternativas. 

### 2.1.1 Alternativa A: Entrenamiento a partir de Imagenet

Se va a entrenar el mismo modelo que el descrito en el apartado anterior, partiendo de los pesos entrenados a partir de ImageNet y no sobre el modelo entrenado con la versión reducida del conjunto de datos. Con esto se pretende descartar que el problema del bajo rendimiento sea el *transfer learning* entre la versión reducida y la versión completa. 

In [21]:
# Alternativa A. Mismos parámetros que el modelo original, entrenado directamente a partir 
# de Imagenet y no a partir de la versión reducida del conjunto de datos. 
def build_VGG16a():
    base_model = VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(224,224,3),
        pooling='max',
        #classes=1000
        )
    freeze_all_layers(base_model)
    
    x = base_model.output
    x = Dense(496, activation="relu")(x)
    x = Dense(NUM_CLASSES, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=x)
    
    return model


%cache coarse_vgg162_a = train('VGG16a',model_id='coarse3', freeze_layers_ratio=0.80)
%cache coarse_vgg162_a_test_res = evaluate('VGG16a', model_id = 'coarse3')
build_result_df([(coarse_vgg162_a_test_res[1], 'VGG16 alt. A')], k=5)

Loading cached value for variable 'coarse_vgg162_a'. Time since caching: 5 days, 15:12:07.485051
Loading cached value for variable 'coarse_vgg162_a_test_res'. Time since caching: 5 days, 3:28:53.756745


,recall.micro,precision.micro,f1.micro,recall.macro,precision.macro,f1.macro
VGG16 alt. A,0.01,0.01,0.00,0.01,0.00,0.00


El resultado obtenido es el mismo que en el apartado anterior. 

### 2.1.2 Alternativa B: Entrenamiento a partir de Imagenet aumentando el tamaño de las capas finales

Partiendo de los pesos de Imagenet, se emplea la misma arquitectura que en los modelos anteriores multiplicando x10 el número de neuronas en la penúltima capa densa. De esta forma se consigue que la última parte de la red tenga forma de embudo.

In [22]:
# Alternativa B. Mismos parámetros que el modelo original, entrenado directamente a partir 
# de Imagenet y no a partir de la versión reducida del conjunto de datos y aumentando el tamaño de la penúltima capa densa. 
def build_VGG16b():
    base_model = VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(224,224,3),
        pooling='max',
        #classes=1000
        )
    freeze_all_layers(base_model)
    
    x = base_model.output
    x = Dense(4096, activation="relu")(x)
    x = Dense(NUM_CLASSES, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=x)
    
    return model


%cache coarse_vgg162_b = train('VGG16b',model_id='coarse4', freeze_layers_ratio=0.80)
%cache coarse_vgg162_b_test_res = evaluate('VGG16b', model_id = 'coarse4')
build_result_df([(coarse_vgg162_b_test_res[1], 'VGG16 alt. B')], k=5)

Loading cached value for variable 'coarse_vgg162_b'. Time since caching: 5 days, 4:05:29.797986
Loading cached value for variable 'coarse_vgg162_b_test_res'. Time since caching: 5 days, 3:20:32.579847


,recall.micro,precision.micro,f1.micro,recall.macro,precision.macro,f1.macro
VGG16 alt. B,0.01,0.01,0.00,0.01,0.00,0.00


### 2.1.3 Alternativa C: Reducir el número de capas congeladas durante el entrenamiento

Partiendo del modelo entrenado con la versión reducida del conjunto de datos, se disminuye de 80% a 60% el porcentaje de capas congeladas durante la segunda fase de entrenamiento. Con esto se pretende comprobar si el problema del bajo rendimiento es que existe una gran diferencia entre los pesos entrenados a partir de Imagenet y el conjunto de datos actual en estas capas de la red.

In [23]:
%cache coarse_vgg162_60 = train('VGG16',model_id='coarse2', freeze_layers_ratio=0.60)
%cache coarse_vgg162_60_test_res = evaluate('VGG16', model_id = 'coarse2')
build_result_df([(coarse_vgg162_60_test_res[1], 'VGG16 alt. C')], k=5)

Loading cached value for variable 'coarse_vgg162_60'. Time since caching: 8 days, 12:37:20.797001
Loading cached value for variable 'coarse_vgg162_60_test_res'. Time since caching: 5 days, 3:37:16.322829


,recall.micro,precision.micro,f1.micro,recall.macro,precision.macro,f1.macro
VGG16 alt. C,0.00,0.00,0.00,0.01,0.00,0.00


### 2.1.4 Alternativa D: Oversampling y undersampling.

Finalmente, se realiza la misma modificación de la partición de entrenamiento con la que mejores resultados se ha obtenido con la versión reducida del conjunto de datos. Recodemos que esta modificación consistía en reemplazar el 30% de las observaciones de la partición de entrenamiento con atributos mayoritarios por observaciones asociadas a atributos minoritarios. El código y la descripción este método están detallados en el *notebook* de [análisis](http://localhost:8888/notebooks/TFM%20akepa/attribute-detection-in-fashon-images/src/DeepFashion_VersionCompleta_Analisis.ipynb#Opci%C3%B3n-2:-Oversampling-y-undersampling-por-etiqueta)

In [24]:
# Sobreescribimos la función definida anteriormente para emplear el fichero modificado de datos
def build_iterator_dataframe(partition, attr_type=None):
    if partition == 'train':
        partition_attribute_filepath = ANNO_COARSE_PATH + "/oversampled.txt"
    else:
        partition_attribute_filepath = ANNO_COARSE_PATH + "/" + partition + "_attr.txt"
    names = []
    names.append('filename')
    names.extend(attributes)
    data = pd.read_csv(partition_attribute_filepath, sep = ' ', names = names, index_col=False)
    
    # Filtrado por tipo de atributo. Empleado para los clasificadores separados por tipo de atributo. 
    if attr_type is not None:
        columns = attr_type_to_columns[str(attr_type)]
        fst, lst = columns[0], columns[-1]+1
        data = data.iloc[:, fst:lst]
    
    return data

In [25]:
%cache coarse_vgg16_oversampled = train('VGG16',model_id='coarse5', freeze_layers_ratio=0.60)
%cache coarse_vgg16_oversampled_test_res = evaluate('VGG16', model_id = 'coarse5')
build_result_df([(coarse_vgg16_oversampled_test_res[1], 'VGG16 alt. D')], k=5)

Loading cached value for variable 'coarse_vgg16_oversampled'. Time since caching: 12:17:56.561902
Loading cached value for variable 'coarse_vgg16_oversampled_test_res'. Time since caching: 12:09:38.822364


,recall.micro,precision.micro,f1.micro,recall.macro,precision.macro,f1.macro
VGG16 alt. D,0.00,0.00,0.00,0.01,0.00,0.00


# 3. Conclusiones

La conclusión que se obtiene con los resultados mostrados en el apartado anterior es que la clasificación de atributos no es abordable con redes convolucionales simples cuando el número de atributos es alto. El valor de la métrica *top-k recall* obtenido (0.01) es inferior a los modelos considerados estado del arte que emplean este mismo conjunto de datos, que obtienen valores superiores a 0.3. 

Estos modelos son más complejos y suelen realizar la predicción en varios pasos, donde cada uno de estos pasos se basa en la predicción realizada en los pasos anteriores. Por ejemplo, tanto *FashionNet* como el modelo descrito en [https://github.com/fdjingyuan/Deep-Fashion-Analysis-ECCV2018](https://github.com/fdjingyuan/Deep-Fashion-Analysis-ECCV2018)
realizan la predicción en tres fases:
1. Predicción de landmarks
2. Predicción de la categoría de la prenda
3. Predicción de los atributos.

donde, para cada una de las predicciónes, se emplea la información generada en el paso anterior tal y como muestra la siguiente imagen, extraída del repositorio anteriormente mencionado:

![title](../img/network.png)

Dado el pobre rendimiento obtenido con la versión con un único clasificador se ha omitido realizar las pruebas con la versión con un clasificador por tipo de atributo puesto que se asume que los resultados obtenidos serán similares. 